In [ ]:
!curl --remote-name-all https://storage.googleapis.com/danielk-files/farsi-text/merged_files/w2c_merged.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  982M  100  982M    0     0   140M      0  0:00:07  0:00:07 --:--:--  151M


In [2]:
!pip install parsivar
!pip install num2fawords
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 36.2 MB 236 kB/s 
     |████████████████████████████████| 1.5 MB 34.6 MB/s 
  Created wheel for parsivar: filename=parsivar-0.2.3-py3-none-any.whl size=36492972 sha256=fc0abb6760bc820935718eac37577445de88c9d3de128af8b2752bb2dde5d553
  Stored in directory: /root/.cache/pip/wheels/ae/67/7a/49cbf08f64d3f76a26eceaf0e481a40e233f05d4356875cbed
  Created wheel for nltk: filename=nltk-3.4.5-py3-none-any.whl size=1449922 sha256=5caab395ab33def8237d0935b10271ecf8120a37ce12949b7e282358e02e564c
  Stored in directory: /root/.cache/pip/wheels/48/8b/7f/473521e0c731c6566d631b281f323842bbda9bd819eb9a3ead
Successfully built parsivar nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simp

In [3]:
import re
import string

from hazm import *
from num2fawords import words

sentence_tokenizer = SentenceTokenizer()
_normalizer = Normalizer()

chars_to_ignore = [
    ",", "?", ".", "!", "-", ";", ":", '""', "%", "'", '"', "�",
    "#", "!", "؟", "?", "«", "»", "،", "(", ")", "؛", "'ٔ", "٬", 'ٔ', ",", "?",
    ".", "!", "-", ";", ":", '"', "“", "%", "‘", "”", "�", "–", "…", "_", "”", '“', '„',
    'ā', 'š',
    # "ء",
]

# In case of farsi
chars_to_ignore = chars_to_ignore + list(string.ascii_lowercase + string.digits)

chars_to_mapping = {
    'ك': 'ک', 'دِ': 'د', 'بِ': 'ب', 'زِ': 'ز', 'ذِ': 'ذ', 'شِ': 'ش', 'سِ': 'س', 'ى': 'ی',
    'ي': 'ی', 'أ': 'ا', 'ؤ': 'و', "ے": "ی", "ۀ": "ه", "ﭘ": "پ", "ﮐ": "ک", "ﯽ": "ی",
    "ﺎ": "ا", "ﺑ": "ب", "ﺘ": "ت", "ﺧ": "خ", "ﺩ": "د", "ﺱ": "س", "ﻀ": "ض", "ﻌ": "ع",
    "ﻟ": "ل", "ﻡ": "م", "ﻢ": "م", "ﻪ": "ه", "ﻮ": "و", 'ﺍ': "ا", 'ة': "ه",
    'ﯾ': "ی", 'ﯿ': "ی", 'ﺒ': "ب", 'ﺖ': "ت", 'ﺪ': "د", 'ﺮ': "ر", 'ﺴ': "س", 'ﺷ': "ش",
    'ﺸ': "ش", 'ﻋ': "ع", 'ﻤ': "م", 'ﻥ': "ن", 'ﻧ': "ن", 'ﻭ': "و", 'ﺭ': "ر", "ﮔ": "گ",

    # "ها": "  ها", "ئ": "ی",

    "a": " ای ", "b": " بی ", "c": " سی ", "d": " دی ", "e": " ایی ", "f": " اف ",
    "g": " جی ", "h": " اچ ", "i": " آی ", "j": " جی ", "k": " کی ", "l": " ال ",
    "m": " ام ", "n": " ان ", "o": " او ", "p": " پی ", "q": " کیو ", "r": " آر ",
    "s": " اس ", "t": " تی ", "u": " یو ", "v": " وی ", "w": " دبلیو ", "x": " اکس ",
    "y": " وای ", "z": " زد ",
    "\u200c": " ", "\u200d": " ", "\u200e": " ", "\u200f": " ", "\ufeff": " ",
}


def multiple_replace(text, chars_to_mapping):
    pattern = "|".join(map(re.escape, chars_to_mapping.keys()))
    return re.sub(pattern, lambda m: chars_to_mapping[m.group()], str(text))


def remove_special_characters(text, chars_to_ignore_regex):
    text = re.sub(chars_to_ignore_regex, '', text).lower() + " "
    return text


def normalizer(text, chars_to_ignore=chars_to_ignore, chars_to_mapping=chars_to_mapping):
    chars_to_ignore_regex = f"""[{"".join(chars_to_ignore)}]"""
    text = text.lower().strip()

    text = _normalizer.normalize(text)
    text = multiple_replace(text, chars_to_mapping)
    text = remove_special_characters(text, chars_to_ignore_regex)
    text = re.sub(" +", " ", text)
    _text = []
    for word in text.split():
        try:
            word = int(word)
            _text.append(words(word))
        except:
            _text.append(str(word))

    text = " ".join(_text) + " "

    text = text.strip() + " "

    return text

In [ ]:
def load_sentences():
    sentences = []
    with open('w2c_merged.txt', 'r', encoding='utf-8' ) as f:
      for line in f:
        lines = sentence_tokenizer.tokenize(line)
        for l in lines:
            ll = normalizer(l)
            sentences.append(ll)
    return sentences


sents = load_sentences()
with open('w2c_cleaned.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(sents))

In [4]:
%%capture

!sudo apt-get install build-essential libboost-all-dev cmake zlib1g-dev libbz2-dev liblzma-dev
!git clone https://github.com/kpu/kenlm
%cd kenlm
!mkdir build
%cd build
!cmake ..
!make -j 4
!make install
%cd /content/

In [5]:
!/content/kenlm/build/bin/lmplz --text 'w2c_cleaned.txt' --arpa words.arpa --order 4 --discount_fallback --temp_prefix /tmp/

=== 1/5 Counting and sorting n-grams ===
Reading /content/drive/MyDrive/files/w2c_cleaned.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
tcmalloc: large alloc 2173591552 bytes == 0x55aab4590000 @  0x7f56a07f51e7 0x55aab26ea912 0x55aab268562e 0x55aab266441b 0x55aab2650176 0x7f569e98ec87 0x55aab2651cda
tcmalloc: large alloc 8694341632 bytes == 0x55ab35e76000 @  0x7f56a07f51e7 0x55aab26ea912 0x55aab26d993a 0x55aab26da378 0x55aab2664438 0x55aab2650176 0x7f569e98ec87 0x55aab2651cda
****************************************************************************************************
Unigram tokens 129533510 types 796985
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:9563820 2:1852514944 3:3473465600 4:5557544960
tcmalloc: large alloc 5557551104 bytes == 0x55aab4590000 @  0x7f56a07f51e7 0x55aab26ea912 0x55aab26d993a 0x55aab26da378 0x55aab2664a07 0x55aab2650176 0x7f569e98ec87 0x55aab2651cda
tcmalloc: large alloc 18525

In [6]:
with open("words.arpa", "r") as read_file, open("words_correct.arpa", "w") as write_file:
  has_added_eos = False
  for line in read_file:
    if not has_added_eos and "ngram 1=" in line:
      count=line.strip().split("=")[-1]
      write_file.write(line.replace(f"{count}", f"{int(count)+1}"))
    elif not has_added_eos and "<s>" in line:
      write_file.write(line)
      write_file.write(line.replace("<s>", "</s>"))
      has_added_eos = True
    else:
      write_file.write(line)

In [7]:
!pip install Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 8.6 MB/s 
     |████████████████████████████████| 2.2 MB 47.5 MB/s 


In [8]:
#!/usr/bin/env python
# ----------------------------------------------------------------------------
# Copyright 2015-2016 Nervana Systems Inc.
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ----------------------------------------------------------------------------
# Modified to support pytorch Tensors

import Levenshtein as Lev
import torch
from six.moves import xrange


class Decoder(object):
    """
    Basic decoder class from which all other decoders inherit. Implements several
    helper functions. Subclasses should implement the decode() method.
    Arguments:
        labels (list): mapping from integers to characters.
        blank_index (int, optional): index for the blank '_' character. Defaults to 0.
        space_index (int, optional): index for the space ' ' character. Defaults to 28.
    """

    def __init__(self, labels, blank_index=0):
        self.labels = labels
        self.int_to_char = dict([(i, c) for (i, c) in enumerate(labels)])
        self.blank_index = blank_index
        space_index = len(labels)  # To prevent errors in decode, we add an out of bounds index for the space
        if ' ' in labels:
            space_index = labels.index(' ')
        self.space_index = space_index

    def wer(self, s1, s2):
        """
        Computes the Word Error Rate, defined as the edit distance between the
        two provided sentences after tokenizing to words.
        Arguments:
            s1 (string): space-separated sentence
            s2 (string): space-separated sentence
        """

        # build mapping of words to integers
        b = set(s1.split() + s2.split())
        word2char = dict(zip(b, range(len(b))))

        # map the words to a char array (Levenshtein packages only accepts
        # strings)
        w1 = [chr(word2char[w]) for w in s1.split()]
        w2 = [chr(word2char[w]) for w in s2.split()]

        return Lev.distance(''.join(w1), ''.join(w2))

    def cer(self, s1, s2):
        """
        Computes the Character Error Rate, defined as the edit distance.
        Arguments:
            s1 (string): space-separated sentence
            s2 (string): space-separated sentence
        """
        s1, s2, = s1.replace(' ', ''), s2.replace(' ', '')
        return Lev.distance(s1, s2)

    def decode(self, probs, sizes=None):
        """
        Given a matrix of character probabilities, returns the decoder's
        best guess of the transcription
        Arguments:
            probs: Tensor of character probabilities, where probs[c,t]
                            is the probability of character c at time t
            sizes(optional): Size of each sequence in the mini-batch
        Returns:
            string: sequence of the model's best guess for the transcription
        """
        raise NotImplementedError


class BeamCTCDecoder(Decoder):
    def __init__(self, labels, lm_path=None, alpha=0, beta=0, cutoff_top_n=40, cutoff_prob=1.0, beam_width=100,
                 num_processes=4, blank_index=0):
        super(BeamCTCDecoder, self).__init__(labels)
        try:
            from ctcdecode import CTCBeamDecoder
        except ImportError:
            raise ImportError("BeamCTCDecoder requires paddledecoder package.")
        labels = list(labels)  # Ensure labels are a list before passing to decoder
        self._decoder = CTCBeamDecoder(labels, lm_path, alpha, beta, cutoff_top_n, cutoff_prob, beam_width,
                                       num_processes, blank_index, log_probs_input=True)

    def convert_to_strings(self, out, seq_len):
        results = []
        for b, batch in enumerate(out):
            utterances = []
            for p, utt in enumerate(batch):
                size = seq_len[b][p]
                if size > 0:
                    transcript = ''.join(map(lambda x: self.int_to_char[x.item()], utt[0:size]))
                else:
                    transcript = ''
                utterances.append(transcript)
            results.append(utterances)
        return results

    def convert_tensor(self, offsets, sizes):
        results = []
        for b, batch in enumerate(offsets):
            utterances = []
            for p, utt in enumerate(batch):
                size = sizes[b][p]
                if sizes[b][p] > 0:
                    utterances.append(utt[0:size])
                else:
                    utterances.append(torch.tensor([], dtype=torch.int))
            results.append(utterances)
        return results

    def decode(self, probs, sizes=None):
        """
        Decodes probability output using ctcdecode package.
        Arguments:
            probs: Tensor of character probabilities, where probs[c,t]
                            is the probability of character c at time t
            sizes: Size of each sequence in the mini-batch
        Returns:
            string: sequences of the model's best guess for the transcription
        """
        probs = probs.cpu()
        out, scores, offsets, seq_lens = self._decoder.decode(probs, sizes)

        strings = self.convert_to_strings(out, seq_lens)
        offsets = self.convert_tensor(offsets, seq_lens)
        return strings, offsets


class GreedyDecoder(Decoder):
    def __init__(self, labels, blank_index=0):
        super(GreedyDecoder, self).__init__(labels, blank_index)

    def convert_to_strings(self, sequences, sizes=None, remove_repetitions=False, return_offsets=False):
        """Given a list of numeric sequences, returns the corresponding strings"""
        strings = []
        offsets = [] if return_offsets else None
        for x in xrange(len(sequences)):
            seq_len = sizes[x] if sizes is not None else len(sequences[x])
            string, string_offsets = self.process_string(sequences[x], seq_len, remove_repetitions)
            strings.append([string])  # We only return one path
            if return_offsets:
                offsets.append([string_offsets])
        if return_offsets:
            return strings, offsets
        else:
            return strings

    def process_string(self, sequence, size, remove_repetitions=False):
        string = ''
        offsets = []
        for i in range(size):
            char = self.int_to_char[sequence[i].item()]
            if char != self.int_to_char[self.blank_index]:
                # if this char is a repetition and remove_repetitions=true, then skip
                if remove_repetitions and i != 0 and char == self.int_to_char[sequence[i - 1].item()]:
                    pass
                elif char == self.labels[self.space_index]:
                    string += ' '
                    offsets.append(i)
                else:
                    string = string + char
                    offsets.append(i)
        return string, torch.tensor(offsets, dtype=torch.int)

    def decode(self, probs, sizes=None):
        """
        Returns the argmax decoding given the probability matrix. Removes
        repeated elements in the sequence, as well as blanks.
        Arguments:
            probs: Tensor of character probabilities from the network. Expected shape of batch x seq_length x output_dim
            sizes(optional): Size of each sequence in the mini-batch
        Returns:
            strings: sequences of the model's best guess for the transcription on inputs
            offsets: time step per character predicted
        """
        _, max_probs = torch.max(probs, 2)
        strings, offsets = self.convert_to_strings(max_probs.view(max_probs.size(0), max_probs.size(1)), sizes,
                                                   remove_repetitions=True, return_offsets=True)
        return strings, offsets

In [9]:
!pip install folium==0.2.1
!pip install https://github.com/kpu/kenlm/archive/master.zip pyctcdecode
!pip install git+https://github.com/huggingface/transformers.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 4.5 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79808 sha256=58ca728794c31302a1bfe24c684ba3fa69e445e53e998e5f2ad0b593bee3524f
  Stored in directory: /root/.cache/pip/wheels/9a/f0/3a/3f79a6914ff5affaf50cabad60c9f4d565283283c97f0bdccf
Successfully built folium
  Attempting uninstall: folium
    Found existing installation: folium 0.12.1.post1
    Uninstalling folium-0.12.1.post1:
      Successfully uninstalled folium-0.12.1.post1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.17.5 requires folium>=0.9.1, but you have folium 0.2.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |█████████████████

In [11]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, AutoProcessor, Wav2Vec2ProcessorWithLM, Wav2Vec2CTCTokenizer

In [12]:
model_name_or_path = "m3hrdadfi/wav2vec2-large-xlsr-persian-v2"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(model_name_or_path, device)

processor = AutoProcessor.from_pretrained(model_name_or_path)
model = Wav2Vec2ForCTC.from_pretrained(model_name_or_path).to(device)

m3hrdadfi/wav2vec2-large-xlsr-persian-v2 cpu


Downloading:   0%|          | 0.00/158 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/138 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:370: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Downloading:   0%|          | 0.00/398 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

In [13]:
vocab_dict = processor.tokenizer.get_vocab()
sort_vocab = sorted((value, key) for (key,value) in vocab_dict.items())

vocab = []
for _, token in sort_vocab:
    vocab.append(token.lower())

vocab[vocab.index(processor.tokenizer.word_delimiter_token)] = ' '

In [14]:
import json

with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [15]:
tokenizer = Wav2Vec2CTCTokenizer(
    "/content/vocab.json", 
    bos_token="<s>",
    eos_token="</s>",
    unk_token="<unk>",
    pad_token="<pad>",
    word_delimiter_token="|",
    do_lower_case=False
)

In [16]:
!/content/kenlm/build/bin/build_binary -T -s trie words_correct.arpa lm4.binary
!rm words_correct.arpa

Reading words_correct.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
tcmalloc: large alloc 1461977088 bytes == 0x55739376e000 @  0x7f5e947591e7 0x55738ea56b8e 0x55738ea4b890 0x55738ea28adb 0x55738ea2b6a4 0x55738ea200c5 0x7f5e92f9dc87 0x55738ea2083a
****************************************************************************************************
Identifying n-grams omitted by SRI
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Writing trie
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


In [17]:
from pyctcdecode import build_ctcdecoder

decoder = build_ctcdecoder(
    labels=vocab,
    kenlm_model_path="lm4.binary",
)

In [18]:
processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=tokenizer,
    decoder=decoder
)

In [19]:
!git clone --recursive https://github.com/parlance/ctcdecode.git
!cd ctcdecode && pip install .

Cloning into 'ctcdecode'...
remote: Enumerating objects: 1102, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 1102 (delta 16), reused 32 (delta 14), pack-reused 1063
Receiving objects: 100% (1102/1102), 782.27 KiB | 2.47 MiB/s, done.
Resolving deltas: 100% (529/529), done.
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 'third_party/kenlm' (https://github.com/kpu/kenlm.git) registered for path 'third_party/kenlm'
Cloning into '/content/ctcdecode/third_party/ThreadPool'...
remote: Enumerating objects: 82, done.        
remote: Total 82 (delta 0), reused 0 (delta 0), pack-reused 82        
Cloning into '/content/ctcdecode/third_party/kenlm'...
remote: Enumerating objects: 14102, done.        
remote: Counting objects: 100% (415/415), done.        
remote: Compressing objects: 100% (289/289), done.        
remote: Total 14102 (delta 127

In [20]:
beam_decoder = BeamCTCDecoder(vocab, lm_path='lm4.binary',
                                # alpha=0.6, beta=0.8,
                                alpha=1.0, beta=3.5,        # fine-tuned
                                cutoff_top_n=80, cutoff_prob=1.0,
                                beam_width=400, num_processes=16,
                                blank_index=vocab.index(processor.tokenizer.pad_token))

In [21]:
!wget https://github.com/aliyzd95/ShEMO-Modification/raw/main/shemo.zip
!unzip shemo.zip
!rm shemo.zip

!wget https://github.com/aliyzd95/ShEMO-Modification/raw/main/modified_shemo.json

!wget https://github.com/pariajm/sharif-emotional-speech-dataset/raw/master/shemo.json

--2022-11-07 16:22:55--  https://github.com/aliyzd95/ShEMO-Modification/raw/main/shemo.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/aliyzd95/ShEMO-Modification/main/shemo.zip [following]
--2022-11-07 16:22:55--  https://media.githubusercontent.com/media/aliyzd95/ShEMO-Modification/main/shemo.zip
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 872402601 (832M) [application/zip]
Saving to: ‘shemo.zip’

shemo.zip           100%[===================>] 831.99M   167MB/s    in 4.9s    

2022-11-07 16:23:20 (169 MB/s) - ‘shemo.zip’ saved [872402601/872402601]

Archive:  shemo.zip


In [22]:
import json

with open('shemo.json', encoding='utf-8') as os:
    original_shemo = json.loads(os.read())

with open('modified_shemo.json', encoding='utf-8') as ms:
    modified_shemo = json.loads(ms.read())

In [23]:
def select_dataset(MODE):

    shemo = []

    if MODE=='modified':
        for name in modified_shemo:
            sample = {}

            modified = modified_shemo[name]
            sample["sentence"] = normalizer(modified["transcript"])
            sample["path"] = modified["path"]
            
            shemo.append(sample) 

    elif MODE=='original':

        import os
        for file in os.listdir('shemo'):
            sample = {}
            file_name = file[:6]

            try:
                original = original_shemo[file_name]
                sample["sentence"] = normalizer(original["transcript"])
                sample["path"] = f'shemo/{file}'
            except:
                continue
            
            shemo.append(sample) 

    else:
        print('MODE must be one of "modified" or "original"')

    return shemo

In [24]:
MODE = 'original'   ### 'original' or 'modified' 
shemo = select_dataset(MODE)
print(f'the {MODE}_ShEMO has {len(shemo)} samples.')

the modified_ShEMO has 3000 samples.


In [25]:
!pip install git+https://github.com/huggingface/datasets.git
!pip install torchaudio
!pip install librosa
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/datasets.git to /tmp/pip-req-build-tohdm0e6
  Running command git clone -q https://github.com/huggingface/datasets.git /tmp/pip-req-build-tohdm0e6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 212 kB 8.9 MB/s 
     |████████████████████████████████| 115 kB 56.4 MB/s 
     |████████████████████████████████| 127 kB 43.0 MB/s 
  Created wheel for datasets: filename=datasets-2.6.2.dev0-py3-none-any.whl size=443863 sha256=4bec72013801827f90169c2ad7ed4b3ca8312f8e3131046e6123375f3fe8285c
  Stored in directory: /tmp/pip-ephem-wheel-cache-doyftz5j/wheels/cc/33/c1/2cacc415b23189a83908e45db67381ba26175ef1e8aa9062aa
Successfully built datasets
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling url

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 7.2 MB/s 
  Attempting uninstall: levenshtein
    Found existing installation: Levenshtein 0.20.8
    Uninstalling Levenshtein-0.20.8:
      Successfully uninstalled Levenshtein-0.20.8


In [26]:
import torchaudio
import librosa
import numpy as np
import pandas as pd
from datasets import load_metric, Dataset

In [27]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    speech_array = speech_array.squeeze().numpy()
    speech_array = librosa.resample(np.asarray(speech_array), sampling_rate, processor_with_lm.feature_extractor.sampling_rate)

    batch["speech"] = speech_array
    return batch

In [28]:
dataset = Dataset.from_pandas(pd.DataFrame(data=shemo))
dataset = dataset.map(speech_file_to_array_fn)

  0%|          | 0/3000 [00:00<?, ?ex/s]

In [29]:
def predict(batch):

    features = processor_with_lm(
        batch["speech"], 
        sampling_rate=processor_with_lm.feature_extractor.sampling_rate, 
        return_tensors="pt", 
        padding=True
    )

    input_values = features.input_values.to(device)
    attention_mask = features.attention_mask.to(device)

    with torch.no_grad():
        logits = model(input_values, attention_mask=attention_mask).logits
    
    beam_decoded_output, beam_decoded_offsets = beam_decoder.decode(logits)

    batch["predicted"] = beam_decoded_output[0][0]
    
    return batch

In [30]:
result = dataset.map(predict, batched=False)

/usr/local/lib/python3.7/dist-packages/dill/_dill.py:1890: PicklingWarning: Pickling a PyCapsule (None) does not pickle any C data structures and could cause segmentation faults or other memory errors when unpickling.
  warnings.warn('Pickling a PyCapsule (%s) does not pickle any C data structures and could cause segmentation faults or other memory errors when unpickling.' % (name,), PicklingWarning)


  0%|          | 0/3000 [00:00<?, ?ex/s]

In [31]:
wer = load_metric("wer")
print(f"WER on {MODE}_ShEMO: {100 * wer.compute(predictions=result['predicted'], references=result['sentence'])}")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


WER on modified_ShEMO: 30.796283813487296


In [32]:
from jiwer import *

count = 0
for data in result:

    target = data["sentence"]
    predict = data["predicted"]

    w = wer(target, predict)
    c = cer(target, predict)

    if w>0.5 and c>0.5:
        count += 1
        print(data["path"])
        print(f'sentence: {target}, predicted: {predict}')
        print(f'wer={w}, cer={c}')
        print('==================================================')

print(f'There are {count} files out of a total of {len(result)} files in the dataset that has wer>0.5 and cer>0.5')

shemo/F01A19.wav
sentence: تو که لب به هیچی نزدی , predicted: که نبی شینا زدیم 
wer=0.8333333333333334, cer=0.5714285714285714
shemo/F01H03.wav
sentence: نگفتم , predicted: نگفتم کن 
wer=1.0, cer=0.6
shemo/F01S04.wav
sentence: دیگه از دستش خسته شدم , predicted: یک حس در بهشت خسته شده 
wer=1.0, cer=0.5238095238095238
shemo/F01S09.wav
sentence: انگار اصلا قلب تو سینه اش نیست , predicted: این کار این قید دوستی ناشی 
wer=1.0, cer=0.6
shemo/F02F01.wav
sentence: نه آقا نه , predicted: نمونه 
wer=1.0, cer=0.6666666666666666
shemo/F03F01.wav
sentence: من قربان , predicted: اما من همونو بقای بن نگه 
wer=2.5, cer=2.25
shemo/F07S01.wav
sentence: بنجی بیچاره من , predicted: و این چیه بیچ هیات به 
wer=2.0, cer=1.0
shemo/F07S20.wav
sentence: جورج نه با بی رحمی ازش گرفتم , predicted: چه ارزونه شی با بی روی از هجرت 
wer=0.8571428571428571, cer=0.5714285714285714
shemo/F07W01.wav
sentence: ده دلار , predicted: دونا 
wer=1.0, cer=0.7142857142857143
shemo/F16F01.wav
sentence: یه نفر داره زنگ می زنه , pre